In [36]:
import pandas as pd
import numpy as np
import os

gicc05 = pd.read_csv('/Users/quinnmackay/Documents/GitHub/BICC/Data Storage/Chronologies/GICC05_time_scale.tab', 
                     sep='\t', skiprows=41, usecols=[0,2,3, 6], names=['age', 'ngrip1_depth', 'ngrip2_depth', 'sigma'])

gicc21 = pd.read_excel('/Users/quinnmackay/Documents/GitHub/BICC/Data Storage/Chronologies/GICC21.xlsx',
                        skiprows=1, sheet_name=1, usecols=[0,1,5,6], names=['age', 'age_unc', 'ngrip1_depth', 'ngrip2_depth'])

gicc21['depth'] = np.where(gicc21['age'] <= 730, gicc21['ngrip1_depth'], gicc21['ngrip2_depth'])

gicc05['depth'] = np.where(gicc05['age'] <= 9527, gicc05['ngrip1_depth'], gicc05['ngrip2_depth'])

gicc05


,age,ngrip1_depth,ngrip2_depth,sigma,depth
0,4,0.350,NaN,1,0.350
1,5,0.950,NaN,1,0.950
2,6,1.475,NaN,1,1.475
3,7,1.975,NaN,1,1.975
4,8,2.500,NaN,1,2.500
...,...,...,...,...,...
60194,60198,NaN,2428.758,2611,2428.758
60195,60199,NaN,2428.768,2611,2428.768
60196,60200,NaN,2428.776,2611,2428.776
60197,60201,NaN,2428.784,2611,2428.784


In [37]:
gicc21.reset_index(drop=True, inplace=True)
gicc21_new = gicc21[['depth', 'age', 'age_unc']]
gicc21_new.dropna(inplace=True)
gicc21_new['comment'] = np.nan

print(gicc21_new)

gicc21_new.to_csv('/Users/quinnmackay/Desktop/temp/ice_age_horizons.txt', index=False, sep="\t")

        depth   age  age_unc  comment
26      9.900    27    2.000      NaN
27     10.257    28    2.000      NaN
28     10.647    29    2.000      NaN
29     10.919    30    2.000      NaN
30     11.226    31    2.000      NaN
...       ...   ...      ...      ...
3239  581.911  3240    6.998      NaN
3240  582.000  3241    6.999      NaN
3241  582.130  3242    7.000      NaN
3242  582.240  3243    7.001      NaN
3243  582.400  3244    7.002      NaN

[3218 rows x 4 columns]


/var/folders/cd/1mh45k3j19g5g_kxkz_17xvc0000gn/T/ipykernel_5086/281022359.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gicc21_new.dropna(inplace=True)
/var/folders/cd/1mh45k3j19g5g_kxkz_17xvc0000gn/T/ipykernel_5086/281022359.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gicc21_new['comment'] = np.nan


In [38]:
#filtering out depths less than covered in gicc21
gicc05 = gicc05[gicc05['depth'] > gicc21['depth'].max()]
gicc05.reset_index(drop=True, inplace=True)
gicc05 = gicc05[['depth', 'age', 'sigma']]
gicc05.dropna(inplace=True)

In [39]:
# do the calculation

# Define year intervals
min_age = gicc05["age"].min()
max_age = gicc05["age"].max()
intervals = np.arange(min_age, max_age, 100) #set year intervals, ignoring the end if less than 100 years

depths = np.interp(intervals, gicc05["age"], gicc05["depth"])

sigma_start = np.interp(intervals, gicc05["age"], gicc05["sigma"])
sigma_end = np.interp(intervals+99, gicc05["age"], gicc05["sigma"])

sigma_diff = sigma_end - sigma_start

#this is here to test for an error
for i in range(len(sigma_diff)):
    if sigma_diff[i] <= 0:
        sigma_diff[i] = 0.1

interval_df = pd.DataFrame({
    "depth_top": depths[:-1],
    "depth_bot": depths[1:],
    "duration": 100,
    "dur_unc": sigma_diff[:-1]
})

interval_df.dropna(inplace=True)

if os.name == 'nt':
    output_file = "E:/GitHub/BICC/Paleochrono/EDML-WDC_test/WDC/ice_age_intervals.txt"
elif os.name == 'posix':
    output_file = '/Users/quinnmackay/Desktop/temp/ice_age_intervals.txt'


interval_df.to_csv(output_file, sep="\t", index=False)

print(f"Done! Saved to: {output_file}")

Done! Saved to: /Users/quinnmackay/Desktop/temp/ice_age_intervals.txt
